In [0]:
"""
/FileStore/tables/deloitte-aug-2021/movies.csv
/FileStore/tables/deloitte-aug-2021/ratings.csv

/FileStore/tables/deloitte-aug-2021/links.csv
/FileStore/tables/deloitte-aug-2021/tags.csv
"""

Out[4]: '\n/FileStore/tables/deloitte-aug-2021/movies.csv\n/FileStore/tables/deloitte-aug-2021/ratings.csv\n\n/FileStore/tables/deloitte-aug-2021/links.csv\n/FileStore/tables/deloitte-aug-2021/tags.csv\n'

In [0]:
# Data Frame: Structured Data, where we have columns (column name, column type) and then data
# data frame: meta data (Schema) + data rows [RDD] + APIs [filter, where, join, agg, etc]

# load csv file using data frame
# spark - Spark Session, Entry point for Spark SQL [DataFrame etc]
# for each driver program, there will be ONLY ONE SPARK CONTEXT
# for each driver program, there can be ONE or MORE SPARK SESSION

# we don't have schema, but spark dataframe csv driver will build one schema for us
# default csv will fix all columns with string type
tagsDf = spark.read.format("csv")\
         .option("header", True)\
         .load('/FileStore/tables/deloitte-aug-2021/tags.csv')

# ***DON't USE THIS WAY
tagsDf.printSchema()
# access schema using schema
tagsDf.show(5) # shows 5 records

root
-- userId: string (nullable = true)
-- movieId: string (nullable = true)
-- tag: string (nullable = true)
-- timestamp: string (nullable = true)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [0]:
# to access schema attached to dataframe
tagsDf.schema

# to access the data rows
# dataframe has rdd, data frame/sql is API, where RDD is core
print(tagsDf.rdd.getNumPartitions())
print(tagsDf.rdd.take(5))


1
[Row(userId='2', movieId='60756', tag='funny', timestamp='1445714994'), Row(userId='2', movieId='60756', tag='Highly quotable', timestamp='1445714996'), Row(userId='2', movieId='60756', tag='will ferrell', timestamp='1445714992'), Row(userId='2', movieId='89774', tag='Boxing story', timestamp='1445715207'), Row(userId='2', movieId='89774', tag='MMA', timestamp='1445715200')]

In [0]:
# load data using inferSchema, request Spark itself to analyse and build a schema with proper data types
# ***DON't USE THIS WAY for LARGE FILES, but OK for small files
linksDf = spark.read.format("csv")\
         .option("header", True)\
         .option("inferSchema", True)\
         .load('/FileStore/tables/deloitte-aug-2021/links.csv')

linksDf.printSchema()
linksDf.show(5)

root
-- movieId: integer (nullable = true)
-- imdbId: integer (nullable = true)
-- tmdbId: integer (nullable = true)

+-------+------+------+
movieId|imdbId|tmdbId|
+-------+------+------+
 1|114709| 862|
 2|113497| 8844|
 3|113228| 15602|
 4|114885| 31357|
 5|113041| 11862|
+-------+------+------+
only showing top 5 rows

In [0]:
# define the schema ourself, better approach
from pyspark.sql.types import StructType, LongType,StringType, IntegerType, DoubleType

movieSchema = StructType()\
              .add("movieId", IntegerType(), True)\
              .add("title", StringType(), True)\
              .add("genres", StringType(), True)


movieDf = spark.read.format("csv")\
         .option("header", True)\
         .schema(movieSchema) \
         .load('/FileStore/tables/deloitte-aug-2021/movies.csv')

movieDf.printSchema()
movieDf.show(4)
 

root
-- movieId: integer (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
+-------+--------------------+--------------------+
only showing top 4 rows

In [0]:
# ratingDf usign schema
# how to avoid using \ for new line continuation with paranthesis

ratingSchema = (StructType()
         .add("userId", IntegerType(), True)
         .add("movieId", IntegerType(), True)
         .add("rating", DoubleType(), True)
         .add("timestamp", LongType(), True))


ratingDf = (spark.read.format("csv")
         .option("header", True)
         .schema(ratingSchema)
         .load('/FileStore/tables/deloitte-aug-2021/ratings.csv'))
 
ratingDf.printSchema()
ratingDf.show(4)

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: long (nullable = true)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
+------+-------+------+---------+
only showing top 4 rows

In [0]:
# Spark has database [NOT ATOMIC, NO INTEGRITY etc], based on Files
# Database [Meta data (database name, table names, table schmea, actual data location) + actual data]
# 4 types tables
#  1. Temp Table [View]
#  2. Global Temp Table [Global Temp view]
#  3. Managed Table [Persisted, managed by Spark, insert/update/delete/select statements]
#  4. External table [Schema is managed by spark, the content is located elsewhere, managed by other  tools like ETL, Glue etc]

In [0]:
spark.sql("SHOW DATABASES").show()
spark.sql("SHOW TABLES").show()

+------------+
databaseName|
+------------+
 default|
+------------+

+--------+---------+-----------+
database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

In [0]:
# create a temp view
# temporary table, available within spark session
# it will be removed once spark session/driver terminated
# SQL API, internally DataSet/RDD, Dataframe
ratingDf.createOrReplaceTempView("ratings")

In [0]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
database|tableName|isTemporary|
+--------+---------+-----------+
 | ratings| true|
+--------+---------+-----------+

In [0]:
# spark.sql returns data frames
df = spark.sql("SELECT * FROM ratings")
df.printSchema()
df.show(2)

root
-- userId: integer (nullable = true)
-- movieId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: long (nullable = true)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows

In [0]:
df = spark.sql("SELECT * FROM ratings")
df.show(5) # ASCII FORMAT PRINT
display(df) # data bricks only, html output, interactive, html table, charts

userId,movieId,rating,timestamp
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931
1,70,3.0,964982400
1,101,5.0,964980868
1,110,4.0,964982176
1,151,5.0,964984041
1,157,5.0,964984100


In [0]:
# MAGIC Function availabel in notebooks jupyter, databricks has one
"""
magic functions starts with %sql %pthon %scala etc 

%sql
 SELECT movieId, rating from ratings
 
" SELECT movieId, rating from ratings" shall be taken as input, put into 
spark.sql ( SELECT movieId, rating from ratings ) and execute and get the result
then displayed using display function (HTML based one)
"""

In [0]:
%sql
-- SQL comment, this sql block executed inside spark.sql() and displayed using display
SELECT * FROM ratings

userId,movieId,rating,timestamp
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931
1,70,3.0,964982400
1,101,5.0,964980868
1,110,4.0,964982176
1,151,5.0,964984041
1,157,5.0,964984100


In [0]:
# store the result of the SQL SELECT statement into another temp view
# CTAS CREATE TEMP TABLE AS SELECT **** from ..
popularMovies = spark.sql("""
CREATE OR REPLACE TEMP VIEW popular_movies AS
SELECT movieId, 
       count(userId) AS total_ratings, 
       avg(rating) AS avg_rating        
FROM
       ratings
GROUP BY movieId
HAVING    total_ratings >= 100 AND     avg_rating >= 3.5
ORDER BY avg_rating DESC
""")

display(popularMovies)

In [0]:
%sql

SHOW TABLES

database,tableName,isTemporary
,popular_movies,true
,ratings,true


In [0]:
# how to get the sql table /temp table as data frame
popularMoviesDf = spark.table("popular_movies") # returns popular_movies dataframe
popularMoviesDf.printSchema()

root
-- movieId: integer (nullable = true)
-- total_ratings: long (nullable = false)
-- avg_rating: double (nullable = true)

In [0]:
movieDf.createOrReplaceTempView('movies')

In [0]:
spark.sql ("""
 CREATE OR REPLACE TEMP VIEW most_popular_movies AS
 SELECT  pm.movieId, movies.title, pm.avg_rating, pm.total_ratings, movies.genres
 FROM   popular_movies pm
 INNER JOIN  movies ON pm.movieId == movies.movieId
""")

Out[26]: DataFrame[]

In [0]:
most_popular_movies = spark.table('most_popular_movies')
most_popular_movies.show(10)
# write to CSV, JSON etc

+-------+--------------------+-----------------+-------------+--------------------+
movieId| title| avg_rating|total_ratings| genres|
+-------+--------------------+-----------------+-------------+--------------------+
 318|Shawshank Redempt...|4.429022082018927| 317| Crime|Drama|
 858|Godfather, The (1...| 4.2890625| 192| Crime|Drama|
 2959| Fight Club (1999)|4.272935779816514| 218|Action|Crime|Dram...|
 1221|Godfather: Part I...| 4.25968992248062| 129| Crime|Drama|
 48516|Departed, The (2006)|4.252336448598131| 107|Crime|Drama|Thriller|
 1213| Goodfellas (1990)| 4.25| 126| Crime|Drama|
 912| Casablanca (1942)| 4.24| 100| Drama|Romance|
 58559|Dark Knight, The ...|4.238255033557047| 149|Action|Crime|Dram...|
 50|Usual Suspects, T...|4.237745098039215| 204|Crime|Mystery|Thr...|
 1197|Princess Bride, T...|4.232394366197183| 142|Action|Adventure|...|
+-------+--------------------+-----------------+-------------+--------------------+
only showing top 10 rows